In [6]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json, os

from pathlib import Path 

from distribution_substation_liang2022.v1.scripts import utils_sub # available at github.com/jieunbyun/network-datasets 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))

row_names = list(edges.keys())

n_comp = len(edges)
n_state = 2 # binary states: working (1) or failed (0)


In [8]:
sys_surv_st = 2
thres_list = [180, 300]  # capacity thresholds: if capacity < thres[i], system state = i: maximum capacity is 360

In [9]:
def sfun_subs(comps_st):
    node_groups = utils_sub.process_nodes(nodes)
    capacity = utils_sub.sys_fun(comps_st, edges, nodes, node_groups, return_details=False)

    sys_st = next((i for i, t in enumerate(thres_list) if t > capacity), len(thres_list))

    return capacity, sys_st, None


# Find rules over thresholds

In [ ]:
pga_str = '0_2g'

with open(rf"data\probs_{pga_str}.json", "r") as f: 
    probs_dict = json.load(f)
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

result = tsum.run_rule_extraction_by_mcs(
    sfun = sfun_subs,
    probs = probs,
    row_names = row_names,
    n_state = n_state,
    output_dir = "tsum_res",
    unk_prob_thres = 1e-3,
    sys_surv_st = sys_surv_st
)



---
Round: 1, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 0, Survival rules: 0, Failure rules: 0
Failure sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 0, System value: 180.0. Total samples: 100000.
New rule (No. of conditions: 2): {'M_TF_2': ('<=', 0), 'M_TF_3': ('<=', 0), 'sys': ('<=', 1)}
Updated sys_vals: [180.0]
---
Round: 2, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 1, Survival rules: 0, Failure rules: 1
Failure sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 0, System value: 216.0. Total samples: 100000.
New rule (No. of conditions: 3): {'M_Line220_6': ('<=', 0), 'M_DS220_9': ('<=', 0), 'M_DS220_10': ('<=', 0), 'sys': ('<=', 1)}
Updated sys_vals: [180.0, 216.0]
---
Round: 3, Unk. prob.: 4.375e-01
Surv probs: 0.000e+00, Fail probs: 5.625e-01
No. of non-dominant rules: 2, Survival rules: 0, Fa

KeyboardInterrupt: 